In [ ]:
from dreye.hardware import dummy_system
from dreye.hardware import dummy_spectrometer
from dreye.core.spectrum_utils import create_gaussian_spectrum
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [ ]:
wls = np.arange(100, 1000, 0.1)
dummy_leds = create_gaussian_spectrum(
    wls, 
    [340, 360, 415, 455, 565, 615], 
    std=8, 
    units='spectralirradiance'
)
plt.plot(dummy_leds.domain, dummy_leds)
plt.xlabel('wavelengths (nm)')
plt.ylabel('irradiance')

In [ ]:
leds = ['duv', 'uv', 'violet', 'rblue', 'lime', 'orange']
channels = ['Dev2/ao1', 'Dev3/ao1', 
            'Dev3/ao0', 'Dev1/ao1', 
            'Dev2/ao0', 'Dev1/ao0']
zeros = [4.4] * 5 + [0.]
maxs = [0.] * 5 + [5.]

In [ ]:
outputs = []
for led, ch, z, m in zip(leds, channels, zeros, maxs):
    outputs.append(
        dummy_system.DummyOutput(
            ch, led, 
            zero_boundary=z, 
            max_boundary=m, 
            units='V'
        )
    )

In [ ]:
system = dummy_system.DummySystem(outputs)

In [ ]:
system

In [ ]:
spec = dummy_spectrometer.Spectrometer(
    wls, 
    dummy_leds, 
    system,
    noise_scale=2, 
)

In [ ]:
from dreye.hardware.measurement_runner import MeasurementRunner

In [ ]:
runner = MeasurementRunner(
    system, 
    spec, 
    wls=None, # np.arange(200, 800, 1), 
    smoothing_window=1, 
    n_steps=10, 
    n_avg=1, 
    remove_zero=False, 
)

In [ ]:
runner.run(verbose=2)

In [ ]:
system.spms.units

In [ ]:
plt.plot(
    system.spms.domain, system.spms
)
plt.xlabel('Volts (V)')
plt.ylabel('photon flux')

In [ ]:
system.spms.map(asarray([30000]*6))

In [ ]:
plt.plot(
    system.spms.normalized_spectrum.wavelengths, 
    system.spms.normalized_spectrum)
plt.xlabel('wavelengths')
plt.ylabel('normalized flux intensity')

In [ ]:
runner.save('measurement_test.json')

In [ ]:
from dreye.io import read_json

In [ ]:
system_loaded = read_json('measurement_test.json')

In [ ]:
print(system_loaded)

In [ ]:
plt.plot(
    system_loaded.spms.domain, system_loaded.spms
)
plt.xlabel('Volts (V)')
plt.ylabel('photon flux')

In [ ]:
plt.plot(
    system_loaded.spms.normalized_spectrum.wavelengths, 
    system_loaded.spms.normalized_spectrum)
plt.xlabel('wavelengths')
plt.ylabel('normalized flux intensity')